In [310]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [311]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [312]:
sheet = sa.open('Wani Spreadsheet')

In [313]:
worksheet = sheet.worksheet('Pesanan')

In [314]:
df = pd.DataFrame(worksheet.get_all_records())

In [315]:
df.rename(columns={'Request cabe makanan utama': 'Cabe'}, inplace=True)

In [316]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,10/04/2023 07:40:06,Rey,Nasi Ayam Bakar,,,,
1,10/04/2023 07:39:45,Angelo,Nasi Ayam Geprek,2,Jamur Crispy,,
2,10/04/2023 07:43:04,Kevin Sander Utomo,Nasi Goreng Spesial,2,,,
3,10/04/2023 08:08:48,Max,Nasi Ayam Geprek,,,,


In [317]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1
df[list] = df[list].replace('', 1)

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Ayam Bakar,1,Rey
1,Nasi Ayam Geprek,2,Angelo
2,Nasi Goreng Spesial,2,Kevin Sander Utomo
3,Nasi Ayam Geprek,1,Max


In [318]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
1,10/04/2023 07:39:45,Angelo,Nasi Ayam Geprek,2,Jamur Crispy,,
3,10/04/2023 08:08:48,Max,Nasi Ayam Geprek,1,,,


In [319]:
# Dataframe non-geprek
df_not_geprek = df[~df['Makanan Utama'].str.contains('Geprek')]

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [320]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-04-10 09:48:42.171405
1 Nasi Ayam Geprek Cabe 2 (Angelo)
1 Nasi Ayam Geprek Cabe 1 (Max)
1 Nasi Ayam Bakar (Rey)
1 Nasi Goreng Spesial (Kevin Sander Utomo)
1 Jamur Crispy (Angelo)


In [321]:
payload = {'message' : '\n'.join(combined_prints)}
r = requests.post('https://notify-api.line.me/api/notify'
                , headers={'Authorization' : 'Bearer {}'.format(token)}
                , params = payload
                , notificationDisabled = True)